# Loading Multi-photon Calcium Imaging Data

Cellular Calcium activity recorded using a multi-photon imaging.

## Relevant ALF objects
* mpci
* mpciROIs
* mpciROITypes
* mpciMeanImage
* mpciStack

## Terminology
* **ROI** - A region of interest, usually a neuron soma, detected using an algorithm such as Suite2P.
* **FOV** - A field of view is a plane or volume covering a region of the brain.
* **Imaging stack** - Multiple FOVs acquired at different depths along the same z axis.

## Finding sessions with imaging data
Sessions that contain any form of imaging data have an 'Imaging' procedure. This includes sessions
photometry, mesoscope, 2P, and widefield data.  To further filter by imaging modality you can query
the imaging type associated with a session's field of view.

```python
# Find mesoscope imaging sessions
import numpy as np

from one.api import ONE
one = ONE()
assert not one.offline, 'ONE must be connect to Alyx for searching imaging sessions'

query = 'field_of_view__imaging_type__name,mesoscope'
eids = one.search(procedures='Imaging', django=query, query_type='remote')
```

Sessions can be further filtered by brain region.  You can filter with by Allen atlas name, acronym
or ID, for example:

* `atlas_name='Primary visual area'`
* `atlas_acronym='VISp'`
* `atlas_id=385`

```python
# Find mesoscope imaging sessions in V1, layer 2/3
query = 'field_of_view__imaging_type__name,mesoscope'
eids = one.search(procedures='Imaging', django=query, query_type='remote', atlas_acronym='VISp2/3')
```

The 'details' flag will return the session details, including a `field_of_view` field which contains
a list of each field of view and its location.  All preprocessed mpci imaging data is in `alf/FOV_XX`
where XX is the field of view number. The `FOV_XX` corresponds to a field of view name in Alyx.

```python
eids, det = one.search(procedures='Imaging', django=query, query_type='remote', atlas_acronym='VISp2/3', details=True)
FOVs = det[0]['field_of_view']
print(FOVs[0])
```

The ibllib AllenAtlas class allows you to search brain region descendents and ancestors in order to
find the IDs of brain regions at a certain granularity.

```python
# Search brain areas by name using Alyx
V1 = one.alyx.rest('brain-regions', 'list', name='Primary visual area')
for area in V1:
    print('%s (%s: %i)' % (area['name'], area['acronym'], area['id']))


from iblatlas.atlas import AllenAtlas
atlas = AllenAtlas()

# Interconvert ID and acronym
V1_id = atlas.regions.acronym2id('VISp')
V1_acronym = atlas.regions.id2acronym(V1_id)

# Show all descendents of primary visual area (i.e. all layers)
atlas.regions.descendants(V1_id)
```

For more information see "[Working with ibllib atlas](../atlas_working_with_ibllib_atlas.html)".

## Loading imaging data for a given field of view

For mesoscope sessions there are likely more than one field of view, not all of which cover the
area of interest.  For mesoscope sessions it's therefore more useful to search by field of view instead.
Each field of view returned contains a session eid for loading data with.

```python
# Search for all mesoscope fields of view containing V1
FOVs = one.alyx.rest('fields-of-view', 'list', imaging_type='mesoscope', atlas_acronym='VISp')
# Download all data for the first field of view
FOV_00  = one.load_collection(FOVs[0]['session'], '*' + FOVs[0]['name'])

# Search the fields of view for a specific session that took place in a given brain region
eid = 'a5550a8e-2484-4539-b7f0-8e5f829d0ba7'
FOVs = one.alyx.rest('fields-of-view', 'list', imaging_type='mesoscope', atlas_id=187, session=eid)
```

## Loading imaging stacks
For mesoscope sessions the same region may be acquired at multiple depths.  The plane at each depth
is considered a separate field of view and are related to one another through the stack object.
If a field of view was acquired as part of a stack, the `stack` field will contain an ID. You can
find all fields of view in a given stack by querying the 'imaging-stack' endpoint:

```python
stack = one.alyx.rest('imaging-stack', 'read', id=FOVs[0]['stack'])
FOVs = stack['slices']
print('There were %i fields of view in stack %s' % (len(FOVs), stack['id']))
```

### List the number of fields of view (FOVs) recorded during a session

```python
from one.api import ONE
one = ONE()
eid = 'b1ca324f-5db7-4106-8be2-0dd9cce17648'

fov_folders = one.list_collections(eid, collection='alf/FOV_*')
fovs = sorted(map(lambda x: int(x[-2:]), fov_folders))
nFOV = len(fovs)
```

## Loading ROI activity for a single session

```python
# Loading ROI activity for a single FOV
ROI_data_00 = one.load_collection(eid, 'alf/FOV_00', object=['mpci', 'mpciROIs', 'mpciROITypes', 'mpciStack'])
print(ROI_data_00.keys())

# Loading ROI activity for all FOVs
all_ROI_data = one.load_collection(eid, 'alf/FOV_*', object=['mpci', 'mpciROIs', 'mpciROITypes', 'mpciStack'])
print(all_ROI_data.keys())
print(all_ROI_data.FOV_00.keys())
```

### Get the brain location of an ROI
The brain location of each ROI are first estimated using the surgical coordinates of the imaging window.
These datasets have an '_estimate' in the name.  After histological alignment, datasets are created
without '_estimate' in the name.  The histologically aligned locations are most accurate and should be
used where available.

```python
roi = 0  # The ROI index to lookup
final_alignment =  'brainLocationsIds_ccf_2017' in ROI_data_00['mpciROI']
key = 'brainLocationsIds_ccf_2017' if final_alignment else 'brainLocationsIds_ccf_2017_estimate'

atlas_id = ROI_data_00['mpciROI'][key][roi]
print(f'ROI {roi} was located in {atlas.regions.id2acronym(atlas_id)}')
```

## Loading times
Timestamps for each frame are in seconds from session start and represent the time when frame acquisition started.
Typically a laser scans each voxel in the field of view in a line by line fashion (this may vary across apparatus and
in configuarations such as dual plane mode).  Thus there is a fixed time offset between regions of interest.
The offset can be found in the mpciStack.timeshift.npy dataset and depending on its shape, may be per voxel or per
scan line.

```python
frame_times = ROI_data_00['mpci']['times']
roi_xyz = ROI_data_00['mpciROIs']['stackPos']
timeshift = ROI_data_00['mpciStack']['timeshift']
roi_offsets = timeshift[roi_xyz[:, len(timeshift.shape)]]
# An array of timestamps of shape (n_roi, n_frames)
roi_times = np.tile(frame_times, (roi_offsets.size, 1)) + roi_offsets[np.newaxis, :].T

import matplotlib.pyplot as plt
roi_signal = ROI_data_00['mpci']['ROIActivityF'].T
roi = 2  # The ROI index to lookup
plt.plot(roi_times[roi], roi_signal[roi])
plt.xlabel('Timestamps / s'), plt.ylabel('ROI activity / photodetector units')
```

### Search for sessions with multi-depth fields of view (imaging stacks)

```python
query = 'field_of_view__stack__isnull,False'
eids, det = one.search(procedures='Imaging', django=query, query_type='remote', details=True)
```

### Search sessions with GCaMP mice
...

## More details
* [Description of mesoscope datasets](https://docs.google.com/document/d/1OqIqqakPakHXRAwceYLwFY9gOrm8_P62XIfCTnHwstg/edit#heading=h.nvzaz0fozs8h)
* [Loading raw mesoscope data](./loading_raw_mesoscope_data.ipynb)